<a href="https://colab.research.google.com/github/SanchitaMahajan/Programming-Assignment/blob/main/Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**ASSIGNMENT**

##By Sanchita Mahajan (20467876)

1. This notebook will prompt the user to input a company's stock market ticker symbol. 
2. It will then get the data from the Yahoo Finance website to extract information on the key of the company. 
3. Then the income tax will be calculated for each
executive based on the ATO tax tables. 
4. The User can further choose if the data
will be saved as a CSV and/or stored in an SQLite database.

##***Here is the link of the Yahoo Finance WebPage:-***

https://au.finance.yahoo.com/







##***Here is the ATO tables based on which Income tax of all Key Executives will be calculated:-***


[Resident tax rates 2021–22
Taxable income](https://www.ato.gov.au/rates/individual-income-tax-rates/)

Income            | Tax on this income 
------------------|----------------------
0 – \$18,200       | Nil
\$18,201 – \$45,000 | 19 cents for each \$1 over \$18,200
\$45,001 – \$120,000 | \$5,092 plus 32.5 cents for each \$1 over \$45,000
\$120,001 – \$180,000 |  \$29,467 plus 37 cents for each \$1 over \$120,000
\$180,001 and over | \$51,667 plus 45 cents for each \$1 over \$180,000

# Initial Algorithm

1. Use input ticker symbol
2. Use ticker to get profile page
3. Copy the table
4. For each row of the table:-
 * Process a row to extract pay
 * Find last character to determine multiplier
 * Calculate income tax

 Note: 

 1. Need to validate the ticker, but for now i assume that user type in a correct ticker.
 2. Still need to work with where to save the income tax

In [1]:
# Install unique packages
!pip install kora -q

# load packages
from bs4 import BeautifulSoup
from kora.selenium import wd
import pandas as pd


     |████████████████████████████████| 57 kB 4.1 MB/s 
     |████████████████████████████████| 56 kB 3.7 MB/s 


In [9]:
def get_url():
  '''This function will prompt the user to input a ticker symbol and get 
  the URL of the profile page'''
  
  ticker_symbol = input( "Please input a ticker symbol ")
  print(ticker_symbol)
  example_url = 'https://au.finance.yahoo.com/quote/{}/profile?p={}'
  result = example_url.format(ticker_symbol, ticker_symbol)
  return result

In [10]:
def get_cells_data(row):
  ''' This function will get the data of the first cells(Name, Title, Pay 
  of the Key Executives''' 
  
  cells = row.find_all('td')
  name = cells[0].text
  title = cells[1].text
  pay = cells[2].text
  return {'name': name, 'title': title,'pay': pay}



In [26]:
def get_extract_records(soup):
  '''This function will extract records of key executives of different 
 ticker symbols'''
 
  table = soup.find('table')
  body = table.find('tbody')
  rows = body.find_all('tr')
  
  extract_records = []
  for row in rows:
    result = get_cells_data(row)
    extract_records.append(result)
  return extract_records

In [19]:
def get_salary(rec):
  ''' This function will get the salary(pay) of the Key executives'''
  
  pay = rec['pay']
  last_pay_character = pay[-1]
  num_part = pay[0:-1]

  if last_pay_character == 'M':
    salary = float(num_part) * 1000000
    #print("Millions")
  elif last_pay_character == 'k':
    salary = float(num_part) * 1000
    #print("Thousand")
  else:
    salary = 0
    #print("Not sure")

  return salary

In [20]:
def calculate_tax(income):
   ''' This function will calculate the tax (based on ATO table) 
   on salary(pay) of the Key executives'''
   
   if income <= 18200:
    tax_payable = 0
   elif 18200 < income <= 45000:
    tax_payable = 0.19 * (income - 18200)
   elif 45000 < income < 120000:
    tax_payable = 5092 + 0.32 * (income - 45000)
   elif 120001 < income < 180000:
    tax_payable = 29467 + 0.37 * (income - 120000)
   elif income > 180000:
    tax_payable = 51667 + 0.45 * (income - 180000)


   return tax_payable

In [52]:
# Get URL
url = get_url()
wd.get(url)
soup = BeautifulSoup(wd.page_source, "html.parser")
records = get_extract_records(soup)

for record in records:
  result = get_salary(record)
  income_tax = calculate_tax(result)
  record['tax'] = income_tax

# creat a pandas datafrme (because recall in a earlier notebok a data frame is easy to save as csv)

execs_df = pd.DataFrame.from_records(records)
execs_df

Please input a ticker symbol NAB.AX
Pl ease input a valid ticker symbol


MaxRetryError: ignored